In [1]:
import os
import wget
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astroquery import sha
import requests
import pyvo as vo




/opt/homebrew/anaconda3/lib/python3.8/site-packages/astroquery/sha/__init__.py:14: UserWarning: Experimental: SHA has not yet been refactored to have its API match the rest of astroquery.
  warnings.warn("Experimental: SHA has not yet been refactored to have its "


In [2]:
#TODO: ESTABLISH DICTIONARY OF PLANET NAMES
#      CENTRALIZE TABLES

In [4]:
 systems_df = pd.read_csv('data/PS_2021.10.21_16.42.26.csv', sep=',', low_memory=False)
 transitspec_df = pd.read_csv('data/transitspec_2021.10.21_17.04.26.csv', sep=',', low_memory=False)
 emispec_df = pd.read_csv('data/emissionspec_2021.10.22_13.36.00.csv', sep=',', low_memory=False)

In [8]:
#what are the MOST e
for col in systems_df.columns:
    print(col)

rowid
pl_name
hostname
pl_letter
hd_name
hip_name
tic_id
gaia_id
default_flag
sy_snum
sy_pnum
sy_mnum
cb_flag
discoverymethod
disc_year
disc_refname
disc_pubdate
disc_locale
disc_facility
disc_telescope
disc_instrument
rv_flag
pul_flag
ptv_flag
tran_flag
ast_flag
obm_flag
micro_flag
etv_flag
ima_flag
dkin_flag
soltype
pl_controv_flag
pl_refname
pl_orbper
pl_orbpererr1
pl_orbpererr2
pl_orbperlim
pl_orbsmax
pl_orbsmaxerr1
pl_orbsmaxerr2
pl_orbsmaxlim
pl_rade
pl_radeerr1
pl_radeerr2
pl_radelim
pl_radj
pl_radjerr1
pl_radjerr2
pl_radjlim
pl_masse
pl_masseerr1
pl_masseerr2
pl_masselim
pl_massj
pl_massjerr1
pl_massjerr2
pl_massjlim
pl_msinie
pl_msinieerr1
pl_msinieerr2
pl_msinielim
pl_msinij
pl_msinijerr1
pl_msinijerr2
pl_msinijlim
pl_cmasse
pl_cmasseerr1
pl_cmasseerr2
pl_cmasselim
pl_cmassj
pl_cmassjerr1
pl_cmassjerr2
pl_cmassjlim
pl_bmasse
pl_bmasseerr1
pl_bmasseerr2
pl_bmasselim
pl_bmassj
pl_bmassjerr1
pl_bmassjerr2
pl_bmassjlim
pl_bmassprov
pl_dens
pl_denserr1
pl_denserr2
pl_denslim
pl_or

In [18]:
earthmass = systems_df['pl_masse']

i =0
for emass_i in earthmass:
    
    if (emass_i >0):
        print(systems_df['pl_name'][i])
        print(emass_i)
    i =i+1

1RXS J160929.1-210524 b
2543.0
1RXS J160929.1-210524 b
4000.0
1RXS J160929.1-210524 b
3000.0
1RXS J160929.1-210524 b
3000.0
2MASS J01033563-5515561 AB b
4131.79
2MASS J01225093-2439505 b
7786.5
2MASS J02192210-3925225 b
4417.837
2MASS J04414489+2301513 b
2383.6
2MASS J12073346-3932539 b
1271.0
2MASS J12073346-3932539 b
1589.15
2MASS J19383260+4603591 b
603.877
2MASS J21402931+1625183 A b
6657.48
2MASS J22362452+4751425 b
3972.875
51 Eri b
635.66
51 Peg b
146.2018
55 Cnc b
254.25
55 Cnc b
263.9785
55 Cnc b
266.966
55 Cnc c
54.4738
55 Cnc c
56.6985
55 Cnc d
1226.77
55 Cnc d
1232.493
55 Cnc e
8.09479
55 Cnc e
8.08
55 Cnc e
8.13928
55 Cnc e
7.81
55 Cnc e
8.08
55 Cnc e
7.74
55 Cnc e
7.99
55 Cnc e
8.37
55 Cnc e
8.63
55 Cnc f
44.812
55 Cnc f
47.0051
70 Vir b
2380.45
AB Pic b
4290.5
AU Mic b
17.0
AU Mic c
13.6
BD+20 594 b
16.3
BD+20 594 b
22.2481
CFBDSIR J145829+101343 b
3337.1
CFHTWIR-Oph 98 b
2479.06157
CHXR 73 b
3994.486
CI Tau b
3686.828
COCONUTS-2 b
2002.31896
CT Cha b
5403.0
CoRoT-1 b
34

Kepler-1647 b
483.0
Kepler-1654 b
158.915
Kepler-1655 b
5.0
Kepler-1656 b
48.6
Kepler-1658 b
1868.8404
Kepler-1661 b
17.0
Kepler-17 b
743.7222
Kepler-17 b
785.0401
Kepler-17 b
778.65
Kepler-170 b
82.6
Kepler-1704 b
1318.98789
Kepler-176 c
23.0
Kepler-176 d
15.2
Kepler-177 b
2.0
Kepler-177 b
5.84
Kepler-177 b
5.4
Kepler-177 c
7.5
Kepler-177 c
14.7
Kepler-177 c
13.5
Kepler-18 b
6.9
Kepler-18 c
18.4
Kepler-18 c
17.3
Kepler-18 d
15.7
Kepler-18 d
14.9
Kepler-18 d
16.4
Kepler-183 c
35.7
Kepler-184 c
8.8
Kepler-189 c
22.7
Kepler-19 b
8.4
Kepler-19 b
20.3
Kepler-19 c
1906.9
Kepler-19 c
13.1
Kepler-19 d
22.5
Kepler-197 c
5.3
Kepler-20 b
8.7
Kepler-20 b
9.7
Kepler-20 c
16.1
Kepler-20 c
12.75
Kepler-20 d
20.1
Kepler-20 d
10.07
Kepler-20 e
3.08
Kepler-20 f
14.3
Kepler-203 c
749.9
Kepler-203 d
33.9
Kepler-21 b
5.08
Kepler-21 b
10.4
Kepler-211 c
18.3
Kepler-215 d
23.6
Kepler-219 d
19.1
Kepler-22 b
36.0
Kepler-221 c
15.1
Kepler-223 b
7.4
Kepler-223 b
4.0
Kepler-223 c
5.1
Kepler-223 c
12.4
Kepler-223 

3213.2613
WASP-18 b
3241.7
WASP-18 b
3314.82
WASP-18 b
3635.9752
WASP-18 b
3241.866
WASP-18 b
3270.4707
WASP-18 c
55.2
WASP-180 A b
286.047
WASP-181 b
95.03117
WASP-182 b
47.03884
WASP-183 b
159.55066
WASP-184 b
181.1631
WASP-185 b
311.4734
WASP-186 b
1341.2426
WASP-187 b
254.264
WASP-19 b
362.3262
WASP-19 b
367.72931
WASP-19 b
371.21
WASP-19 b
354.06262
WASP-19 b
370.27195
WASP-19 b
339.76027
WASP-19 b
356.92309
WASP-190 b
317.83
WASP-192 b
731.009
WASP-2 b
269.20201
WASP-2 b
295.89973
WASP-2 b
290.484
WASP-2 b
290.802
WASP-2 b
279.6904
WASP-2 b
279.68
WASP-2 b
269.20201
WASP-2 b
293.99275
WASP-2 b
275.24078
WASP-2 b
291.76794
WASP-20 b
120.13974
WASP-20 b
98.841
WASP-20 b
98.84513
WASP-21 b
92.1707
WASP-21 b
95.345
WASP-21 b
93.75985
WASP-21 b
94.39551
WASP-22 b
212.9461
WASP-22 b
181.1631
WASP-22 b
177.9848
WASP-22 b
180.84527
WASP-22 b
186.876
WASP-22 b
196.10111
WASP-23 b
280.95
WASP-23 b
279.37257
WASP-24 b
394.1092
WASP-24 b
340.382
WASP-24 b
336.8998
WASP-24 b
352.47347
WASP-24